In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# read in metacritic data
df_meta = pd.read_csv('metacritic.csv', header=0)

In [14]:
# preview data
df_meta.head()

# rename columns
df_meta = df_meta.rename(columns={"release_data": "release_date", "tittle": "title", "plataforma": "platform"})

df_meta.head()

,Unnamed: 0,release_date,title,userscore,metascore,platform,genre,developer,publisher,rating
0,0,1998-11-23,The Legend of Zelda: Ocarina of Time,9.1,99.0,Nintendo 64,"Action Adventure, ...",Nintendo,...,E
1,1,2000-09-20,Tony Hawk's Pro Skater 2,7.4,98.0,PlayStation,"Sports, ...",Neversoft Entertainment,...,T
2,2,2021-03-30,Disco Elysium: The Final Cut,8.2,97.0,PC,"Role-Playing, ...",ZA/UM,...,M
3,3,2000-10-30,Jet Grind Radio,8.3,94.0,Dreamcast,"Action, ...",Smilebit,...,T
4,4,2021-08-19,Quake Remastered,8.4,87.0,Switch,"Action, ...","id Software, MachineGames",...,M


In [15]:
# drop first column
df_meta = df_meta.iloc[:,1:]

# data types of columns
print(df_meta.dtypes)

# convert metascore from float to int
#df['metascore'] = df['metascore'].fillna(0).astype('int')


release_date     object
title            object
userscore       float64
metascore       float64
platform         object
genre            object
developer        object
publisher        object
rating           object
dtype: object


In [16]:
# pass in numeric groups only (userscore & metascore) to avoid warning message
df_meta.groupby(['platform'])[['userscore','metascore']].mean().round(1).sort_values(by="userscore")

,userscore,metascore
platform,,
PlayStation 5,6.4,74.1
PlayStation 4,6.4,70.5
Xbox One,6.4,72.0
Xbox Series X,6.5,74.9
Xbox 360,6.9,68.7
PC,6.9,71.4
PlayStation 3,6.9,70.4
Wii,7.1,66.1
3DS,7.1,69.3


In [17]:
# webscrape the console year and month to join to the platform field in this data
# reference: https://www.geeksforgeeks.org/scraping-wikipedia-table-with-pandas-using-read_html/
# average userscore over time compared to average metascore overtime, by platform
# conda install lxml
# conda install html5lib

my_table = pd.read_html("https://en.wikipedia.org/wiki/Home_video_game_console",
                         match='Units')
#my_table[0].head()

In [18]:
my_table[0]

,Name,Release date,Manufacturer,Units sold,CPU,"""Bits"""
0,Fairchild Channel F,November 1976,Fairchild (U.S.),"ca. 250,000",Fairchild F8,8-bit (CPU)
1,RCA Studio II,January 1977,RCA (U.S.),"ca. 60,000",RCA 1802,8-bit (CPU)
2,Bally Astrocade,April 1978,Midway (U.S.),?,Zilog Z80,8-bit (CPU)
3,Atari 2600,"September 11, 1977",Atari Inc. (U.S.),ca. 30 million[2],MOS Technology 6507,8-bit (CPU)
4,APF-MP1000,"January 1, 1978",APF (U.S.),"> 50,000",Motorola 6800,8-bit (CPU)
...,...,...,...,...,...,...
97,Nintendo Switch[e],"March 3, 2017",Nintendo (Japan),107.65 million[24],Octa-core (4×ARM Cortex-A57 & 4×ARM Cortex-A53...,64-bit (CPU)
98,Xbox Series X/S,"November 10, 2020",Microsoft (U.S.),ca. 8 million[25][d],".mw-parser-output .plainlist ol,.mw-parser-out...",64-bit (CPU)
99,PlayStation 5,"November 12, 2020",Sony (Japan),13.4 million[28],"Custom 8-core AMD Zen 2, variable frequency, u...",64-bit (CPU)
100,Polymega,"September 12, 2021","Playmaji, Inc (U.S.)",?,Unknown Intel Coffee Lake,64-bit (CPU)


In [20]:
pd.merge(df_meta.rename({'platform': 'Name'}, axis=1), my_table[0], how = "left", on = ['Name'])

,release_date,title,userscore,metascore,Name,genre,developer,publisher,rating,Release date,Manufacturer,Units sold,CPU,"""Bits"""
0,1998-11-23,The Legend of Zelda: Ocarina of Time,9.1,99.0,Nintendo 64,"Action Adventure, ...",Nintendo,...,E,"June 23, 1996",Nintendo (Japan),32.93 million,NEC VR4300 @ 93.75 MHz,64-bit
1,2000-09-20,Tony Hawk's Pro Skater 2,7.4,98.0,PlayStation,"Sports, ...",Neversoft Entertainment,...,T,NaN,NaN,NaN,NaN,NaN
2,2021-03-30,Disco Elysium: The Final Cut,8.2,97.0,PC,"Role-Playing, ...",ZA/UM,...,M,NaN,NaN,NaN,NaN,NaN
3,2000-10-30,Jet Grind Radio,8.3,94.0,Dreamcast,"Action, ...",Smilebit,...,T,"November 27, 1998",Sega (Japan),9.13 million,Hitachi SH-4 32-bit RISC @ 200 MHz,"128-bit (32-bit processor, 128-bit graphics)"
4,2021-08-19,Quake Remastered,8.4,87.0,Switch,"Action, ...","id Software, MachineGames",...,M,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,2017-03-07,Disc Jam,6.3,72.0,PlayStation 4,"Sports, ...",High Horse Entertainment,...,E,"November 15, 2013",Sony (Japan),115.9 million[18],Semi-custom 8-core AMD x86-64 Jaguar 1.6 GHz C...,64-bit (CPU)
15858,2010-09-22,Serious Sam HD: The Second Encounter,7.3,72.0,Xbox 360,"Action, ...",Croteam,...,M,"November 22, 2005",Microsoft (U.S.),ca. 85.8 million[10][11][12][13],Big-endian architecture 3.2 GHz PowerPC Tri-Co...,64-bit CPU 128-bit extensions
15859,2021-02-18,Cathedral,6.1,72.0,Switch,"Action Adventure, ...",Decemberborn Interactive,...,E10+,NaN,NaN,NaN,NaN,NaN
15860,2013-04-02,Cities in Motion 2,7.2,72.0,PC,"Simulation, ...",Colossal Order,...,NaN,NaN,NaN,NaN,NaN,NaN
